# Variational Quantum Classifier Feature Map Comparison

Both the first-order and second-order expansion feature maps provided by Aqua use $n$ qubits to encode $n$-dim datapoints. However, raw feature vectors can also be directly used in `VQC` circuit constructions, requiring only $log_2(n)$ qubits to encode $n$-dim datapoints. 

### Experiment
Below we compare the classification performance of `VQC` on the [Wine dataset](https://scikit-learn.org/stable/datasets/index.html#wine-dataset) using `RawFeatureVector` and `SecondOrderExpansion` feature maps. As you'll see, the former leads to about $90\%$ accuracy using only $2$ qubits, whereas the latter achieves only around $50\%$ accuracy, using $4$ qubits and taking $3\times$ as long. 

We first prepare the Wine dataset:

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from qiskit.ml.datasets import wine

We can then set up the experiment as follows:

In [2]:
import numpy as np
import scipy

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.components.feature_maps import RawFeatureVector, SecondOrderExpansion
from qiskit.aqua.utils import get_feature_dimension

feature_dim = 4 # dimension of each data point
training_dataset_size = 20
testing_dataset_size = 10
random_seed = 10598
aqua_globals.random_seed = random_seed 

sample_Total, training_input, test_input, class_labels = wine(
    training_size=training_dataset_size,
    test_size=testing_dataset_size,
    n=feature_dim,
    plot_data=False
)

Let's try `RawFeatureVector` first:

In [3]:
feature_map = RawFeatureVector(feature_dimension=get_feature_dimension(training_input))
vqc = VQC(COBYLA(maxiter=200),
            feature_map,
            RYRZ(feature_map.num_qubits, depth=3),
            training_input, test_input)
result = vqc.run(QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                    seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed))
print("VQC accuracy with RawFeatureVector: ", result['testing_accuracy'])

VQC accuracy with RawFeatureVector:  0.7


Now let's try `SecondOrderExpansion`:

In [4]:
feature_map = SecondOrderExpansion(feature_dimension=get_feature_dimension(training_input))
vqc = VQC(COBYLA(maxiter=200),
            feature_map,
            RYRZ(feature_map.num_qubits, depth=3),
            training_input, test_input)
result = vqc.run(QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                    seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed))
print("Test accuracy with SecondOrderExpansion: ", result['testing_accuracy'])

Test accuracy with SecondOrderExpansion:  0.5
